In [1]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import transform_gfk, adapt_gfk
from sklearn.model_selection import train_test_split
from bob.learn.linear import GFKTrainer

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import itertools

#variables para guardar los resultados
tipo = pruebas[2]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Pruebas con el dataset Amazon (3000 Dimensiones)

In [2]:
dims = 3000
dataset_name = datasets[0]

In [3]:
print tipo
print dataset_name
print dims
print data_path

gfk
amazon
3000
data


In [4]:
# cargando dataset
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [5]:
parametros = {
    'dims': [int(dims/4), int(dims/2)],
    'n_subs': [10, 20, 50]
}

dim_nsubs_pair =  list(itertools.product(parametros['dims'], parametros['n_subs']))


In [6]:
porcentajes = [25, 50, 100]
modelos = {}

pairs = list(itertools.permutations(domains, 2))

for src, tgt in pairs:
    modelos[(src, tgt)] = {}
    for ptje in porcentajes:
        modelos[(src, tgt)][ptje] = [100, None]

In [7]:
df = pd.DataFrame(columns=dataframe_columns+['Porcentaje'])

i=0
j=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

porcentajes = [25, 50, 100]


for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)
    tarea = src[0]+'->'+tgt[0]
    
    #datos de entrenamiento
    X_src = labeled[src]['X_tr'][:, :dims].todense()
    y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
    
    X_tgt = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tgt = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
    
    #datos de prueba
    X_tgt_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_tgt_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
    
    
    #baseline in-domain error
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)
    
    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tgt, y_tgt)
    b_error = 1-svc.score(X_tgt_ts, y_tgt_ts)

    k = 0
    for dim, n_subs in dim_nsubs_pair:
        print "\tEntrenando modelo de adaptacion %d" % k
        # entrenar un modelo de adaptacion
        modelo_temp = adapt_gfk(X_src, X_tgt, n_subs, dim)

        # entrenar un clasificador con los datos adaptados
        X_src_a = transform_gfk(X_src, modelo_temp)
        clf = get_best_score(X_src_a, y_src, classifier='SVC', n_jobs=4)

        # se prueba con distintos porcentajes
        j = i*3
        for porcentaje in porcentajes:
            if porcentaje == 100:
                X_test_pc, y_test_pc = X_tgt_ts, y_tgt_ts
            else:
                X_test_pc, _, y_test_pc, _ = train_test_split(X_tgt_ts, y_tgt_ts, train_size=porcentaje/float(100))
            
            X_test_pc_a = transform_gfk(X_test_pc, modelo_temp)
            t_error = 1-clf.score(X_test_pc_a, y_test_pc)
            
            if t_error <= modelos[(src, tgt)][porcentaje][0]:
                modelos[(src, tgt)][porcentaje] = [t_error, modelo_temp]
                t_loss = t_error - b_error
                df.loc[j] = ['GFK', tarea, src, tgt, b_error*100, t_error*100, t_loss*100, porcentaje]
    
            j += 1
        k += 1
    i+=1

Tarea 1 de 12
Cargando modelo existente.
	Entrenando modelo de adaptacion 0


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

	Entrenando modelo de adaptacion 1
	Entrenando modelo de adaptacion 2
	Entrenando modelo de adaptacion 3
	Entrenando modelo de adaptacion 4
	Entrenando modelo de adaptacion 5
Tarea 2 de 12
Cargando modelo existente.
	Entrenando modelo de adaptacion 0
	Entrenando modelo de adaptacion 1
	Entrenando modelo de adaptacion 2
	Entrenando modelo de adaptacion 3
	Entrenando modelo de adaptacion 4
	Entrenando modelo de adaptacion 5
Tarea 3 de 12
Cargando modelo existente.
	Entrenando modelo de adaptacion 0
	Entrenando modelo de adaptacion 1
	Entrenando modelo de adaptacion 2
	Entrenando modelo de adaptacion 3
	Entrenando modelo de adaptacion 4
	Entrenando modelo de adaptacion 5
Tarea 4 de 12
Cargando modelo existente.
	Entrenando modelo de adaptacion 0
	Entrenando modelo de adaptacion 1
	Entrenando modelo de adaptacion 2
	Entrenando modelo de adaptacion 3
	Entrenando modelo de adaptacion 4
	Entrenando modelo de adaptacion 5
Tarea 5 de 12
Cargando modelo existente.
	Entrenando modelo de adaptacio

In [8]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss,Porcentaje
0,mSDA,e->d,electronics,dvd,15.762894,17.520000,1.757106,25
1,mSDA,e->d,electronics,dvd,15.762894,21.505376,5.742482,50
2,mSDA,e->d,electronics,dvd,15.762894,24.378109,8.615215,100
3,mSDA,e->k,electronics,kitchen,8.587715,7.670455,-0.917260,25
4,mSDA,e->k,electronics,kitchen,8.587715,8.300830,-0.286885,50
5,mSDA,e->k,electronics,kitchen,8.587715,9.922748,1.335033,100
6,mSDA,e->b,electronics,books,15.157879,23.579545,8.421667,25
7,mSDA,e->b,electronics,books,15.157879,25.472926,10.315047,50
8,mSDA,e->b,electronics,books,15.157879,28.463212,13.305333,100
9,mSDA,d->e,dvd,electronics,11.127778,17.171717,6.043939,25


In [12]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "inductive_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/amazon/gfk/inductive_3000.csv
Resultados guardados.


# Pruebas con el dataset Twitter (2000 Dimensiones)

In [13]:
dims = 2000
dataset_name = datasets[1]

In [14]:
print tipo
print dataset_name
print dims
print data_path

gfk
twitter
2000
data


In [15]:
# cargando dataset
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [16]:
parametros = {
    'dims': [int(dims/4), int(dims/2)],
    'n_subs': [10, 20, 50]
}

dim_nsubs_pair =  list(itertools.product(parametros['dims'], parametros['n_subs']))


In [17]:
porcentajes = [25, 50, 100]
modelos = {}

pairs = list(itertools.permutations(domains, 2))

for src, tgt in pairs:
    modelos[(src, tgt)] = {}
    for ptje in porcentajes:
        modelos[(src, tgt)][ptje] = [100, None]

In [18]:
df = pd.DataFrame(columns=dataframe_columns+['Porcentaje'])

i=0
j=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

porcentajes = [25, 50, 100]


for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)
    tarea = src[0]+'->'+tgt[0]
    
    #datos de entrenamiento
    X_src = labeled[src]['X_tr'][:, :dims].todense()
    y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
    
    X_tgt = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tgt = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
    
    #datos de prueba
    X_tgt_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_tgt_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
    
    
    #baseline in-domain error
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)
    
    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tgt, y_tgt)
    b_error = 1-svc.score(X_tgt_ts, y_tgt_ts)

    k = 0
    for dim, n_subs in dim_nsubs_pair:
        print "\tEntrenando modelo de adaptacion %d" % k
        # entrenar un modelo de adaptacion
        modelo_temp = adapt_gfk(X_src, X_tgt, n_subs, dim)

        # entrenar un clasificador con los datos adaptados
        X_src_a = transform_gfk(X_src, modelo_temp)
        clf = get_best_score(X_src_a, y_src, classifier='SVC', n_jobs=4)

        # se prueba con distintos porcentajes
        j = i*3
        for porcentaje in porcentajes:
            if porcentaje == 100:
                X_test_pc, y_test_pc = X_tgt_ts, y_tgt_ts
            else:
                X_test_pc, _, y_test_pc, _ = train_test_split(X_tgt_ts, y_tgt_ts, train_size=porcentaje/float(100))
            
            X_test_pc_a = transform_gfk(X_test_pc, modelo_temp)
            t_error = 1-clf.score(X_test_pc_a, y_test_pc)
            
            if t_error <= modelos[(src, tgt)][porcentaje][0]:
                modelos[(src, tgt)][porcentaje] = [t_error, modelo_temp]
                t_loss = t_error - b_error
                df.loc[j] = ['GFK', tarea, src, tgt, b_error*100, t_error*100, t_loss*100, porcentaje]
    
            j += 1
        k += 1
    i+=1

Tarea 1 de 6
Cargando modelo existente.
	Entrenando modelo de adaptacion 0
	 25
	 50
	 100
	Entrenando modelo de adaptacion 1
	 25
	 50
	 100
	Entrenando modelo de adaptacion 2
	 25
	 50
	 100
	Entrenando modelo de adaptacion 3
	 25
	 50
	 100
	Entrenando modelo de adaptacion 4
	 25
	 50
	 100
	Entrenando modelo de adaptacion 5
	 25
	 50
	 100
Tarea 2 de 6
Cargando modelo existente.
	Entrenando modelo de adaptacion 0
	 25
	 50
	 100
	Entrenando modelo de adaptacion 1
	 25
	 50
	 100
	Entrenando modelo de adaptacion 2
	 25
	 50
	 100
	Entrenando modelo de adaptacion 3
	 25
	 50
	 100
	Entrenando modelo de adaptacion 4
	 25
	 50
	 100
	Entrenando modelo de adaptacion 5
	 25
	 50
	 100
Tarea 3 de 6
Cargando modelo existente.
	Entrenando modelo de adaptacion 0
	 25
	 50
	 100
	Entrenando modelo de adaptacion 1
	 25
	 50
	 100
	Entrenando modelo de adaptacion 2
	 25
	 50
	 100
	Entrenando modelo de adaptacion 3
	 25
	 50
	 100
	Entrenando modelo de adaptacion 4
	 25
	 50
	 100
	Entrenando m

In [19]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss,Porcentaje
0,GFK,r->t,rio2016,thevoice,7.362689,14.117647,6.754958,25
1,GFK,r->t,rio2016,thevoice,7.362689,15.444015,8.081326,50
2,GFK,r->t,rio2016,thevoice,7.362689,19.673295,12.310606,100
3,GFK,r->g,rio2016,general,26.998127,38.983051,11.984924,25
4,GFK,r->g,rio2016,general,26.998127,38.533986,11.535859,50
5,GFK,r->g,rio2016,general,26.998127,40.684357,13.686230,100
6,GFK,t->r,thevoice,rio2016,10.208711,23.437500,13.228789,25
7,GFK,t->r,thevoice,rio2016,10.208711,21.739130,11.530419,50
8,GFK,t->r,thevoice,rio2016,10.208711,23.502722,13.294011,100
9,GFK,t->g,thevoice,general,26.998127,38.544126,11.545999,25


In [20]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "inductive_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/twitter/gfk/inductive_2000.csv
Resultados guardados.
